<a href="https://colab.research.google.com/github/ahmedhamodien/Machine-Learning-/blob/main/Machine_Learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!git clone https://github.com/your_username/Machine_Learning.git
%cd face-mask-detection


Cloning into 'Machine_Learning'...
fatal: could not read Username for 'https://github.com': No such device or address
[Errno 2] No such file or directory: 'face-mask-detection'
/content


test test